In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [3]:
dataset = 'mater'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,3043802,Fabrication of SUS 304 regularly cell-structur...,2.594487e+09,1800.0,0,,"271362782,1055509267,2477281293,2002957515",145550701.0,"60,693,14",
1,2829099,Solid-state synthesis of Mg-Si-Ge ternary ther...,2.397477e+09,1800.0,0,,"2002957515,2937526791,2940158953",145550701.0,"8076,60,583,14,2318",
2,3070491,Solid-state synthesis of magnesium alloys.,2.608738e+09,1800.0,0,,2652625152,87940674.0,"14,60,935,2318",
3,3045077,"T. Aizawa, R. Song and A. Yamamoto: Solid stat...",2.595067e+09,1800.0,0,,"2002957515,2829560477,2898382323",145550701.0,"583,60,596,2318,14,816",
4,3359671,Plan of Ellis' Island showing proposed improve...,2.857023e+09,1866.0,0,,2812590778,91170297.0,"227,14,496,44380",
...,...,...,...,...,...,...,...,...,...,...
3910962,3888693,Recent progress on control strategies for inhe...,3.035239e+09,2021.0,0,"73799577,102216338,102871714,773428209,7821514...","2552592731,2099021022,2295711622,2597971149,25...",88404446,"11161,136,7391,14,5218,9689,1230,60,5841,25135",
3910963,3841379,Short-Term Aging Performance Evaluation of Asp...,3.019921e+09,2021.0,1,,"2793422944,2792311437,2789302698,2789926085,27...",163152955,"219,1850,14,44699,27452,10557,670,11298,28289,...",
3910964,3876446,The Deterioration of Cement Mortar under the C...,3.031797e+09,2021.0,1,,"3029521519,2465873040",163152955,"3296,14,11940,316,2066,4588,381,330,13,4940",
3910965,3848679,Vibration Levels of Stacked Automotive Engine ...,3.022042e+09,2021.0,0,,2075634521,163152955,"3999,4091,52801,53455,14,1273,219,40272,13273,...",


In [4]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 1061716 rows


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
1060499,2423411,"""Burst"" technology with feedback-loop control ...",2.140178e+09,2000.0,15,"1981996417,2015483304,2022604818,2024323482,20...","2081067593,1910170253,2002783956,2124703562,22...",162355128,"12567,1555,2405,4119,828,2245,2105,80,1675,347,54",
1060500,1006677,"""Growth of Mesoscopic Correlated Droplet Patte...",2.015806e+09,2000.0,41,,"2922856191,2066033401,2179978478,2122616039,29...",6791298,"7062,150,3071,1349,54,5711,73",
1060501,354161,"""Matrix"" effects in ToF-SIMS analyses of styre...",1.973611e+09,2000.0,16,,"2118200988,2152164400,2288649338",177196338,"15,3654,311,1741,3582,5,11,2,2241,3653",
1060502,88172,"""Millipede"" - An AFM data storage system at th...",1.216362e+09,2000.0,45,"633031011,1986838720,2002427746,2006515893,201...","1992287064,2228330870,1227322763,1653105804,20...",116094739,"39655,392,1493,14,13669,139,1015,9111,2071,5353",
1060503,2214306,"""NISSEKI LC Film"" Liquid Crystalline Polymer F...",2.094522e+09,2000.0,1,,"2623456286,2700940211",26060816,"270,227,4,954,14,8578,5730,638,13,392,52",
...,...,...,...,...,...,...,...,...,...,...
3910962,3888693,Recent progress on control strategies for inhe...,3.035239e+09,2021.0,0,"73799577,102216338,102871714,773428209,7821514...","2552592731,2099021022,2295711622,2597971149,25...",88404446,"11161,136,7391,14,5218,9689,1230,60,5841,25135",
3910963,3841379,Short-Term Aging Performance Evaluation of Asp...,3.019921e+09,2021.0,1,,"2793422944,2792311437,2789302698,2789926085,27...",163152955,"219,1850,14,44699,27452,10557,670,11298,28289,...",
3910964,3876446,The Deterioration of Cement Mortar under the C...,3.031797e+09,2021.0,1,,"3029521519,2465873040",163152955,"3296,14,11940,316,2066,4588,381,330,13,4940",
3910965,3848679,Vibration Levels of Stacked Automotive Engine ...,3.022042e+09,2021.0,0,,2075634521,163152955,"3999,4091,52801,53455,14,1273,219,40272,13273,...",


## build graph

In [5]:
dataset = 'mater'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=3907079, dict([2594486982: [-0.25881624  0.11267769  0.09194859 -0.00532467 -0.1772402   0.03327483
 -0.09638989  0.1337313  -0.18410267  0.10506696  0.24807596  0.2189675
  0.43442428 -0.1525583   0.16722907 -0.19770162  0.1267222   0.13658407
  0.02654327  0.01406975 -0.2723631  -0.04185371 -0.2917231  -0.20154338
  0.12882498 -0.14430518  0.07126188  0.27865326 -0.04251712 -0.5864822
 -0.02894852 -0.21035212 -0.01245849  0.01860973  0.22484602  0.30658942
  0.22567296 -0.22688341 -0.03399128  0.09611323  0.04186709 -0.12478792
 -0.03428599  0.09687325 -0.03985746 -0.2212709  -0.26379594  0.3068253
  0.04237314  0.11996776 -0.05647688 -0.05327208  0.04398521 -0.08803191
  0.09720346  0.14527684 -0.14587386 -0.0204276  -0.13436006 -0.11369891
 -0.30549964  0.2813353  -0.3587787  -0.30062607  0.23636204  0.24434876
 -0.06755412  0.04600637 -0.04078185 -0.07094013  0.13205697  0.11449602
  0.28420225  0.11442963 -0.10054141 -0.04883131 -0.10687283 -0.18403219
 -0.24955332

In [6]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 2846619/2846619 [05:58<00:00, 7934.19it/s]


feat : shape=torch.Size([11374968, 128])
tensor([[ 0.1364,  0.0142,  0.0853,  ...,  0.2533,  0.2737, -0.1916],
        [-0.8975, -0.1294, -0.7220,  ..., -0.7287, -0.4320, -0.7714],
        [-0.5973, -0.9251, -0.0298,  ..., -0.3392, -0.8605, -0.4797],
        ...,
        [-0.8289, -0.7524, -0.4830,  ..., -0.0409, -0.7568, -0.4972],
        [-0.0217,  0.2701,  0.1924,  ...,  0.2130,  0.1954, -0.2845],
        [ 0.1057, -0.0227,  0.2440,  ...,  0.2335, -0.0188, -0.0085]])
src : shape=torch.Size([66118536]), tensor([       0,        0,        0,  ..., 11374945, 11374945, 11374945])
tgt : shape=torch.Size([66118536]), tensor([       1,        2,        3,  ...,  9775554, 11374965, 10185482])
rel : shape=torch.Size([66118536]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([66118536]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [7]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=11374968, num_edges=66118536,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [8]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [9]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [10]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 2846619/2846619 [01:44<00:00, 27318.87it/s]


In [11]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 662152, 1391974, ..., 60626235, 66116372, 66118536])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   662152]
 [    2001   662152  1391974]
 [    2002  1391974  2201200]
 [    2003  2201200  3191597]
 [    2004  3191597  4396520]
 [    2005  4396520  5826410]
 [    2006  5826410  7455224]
 [    2007  7455224  9358620]
 [    2008  9358620 11395852]
 [    2009 11395852 13543797]
 [    2010 13543797 15906376]
 [    2011 15906376 19064340]
 [    2012 19064340 22559859]
 [    2013 22559859 26601876]
 [    2014 26601876 31003887]
 [    2015 31003887 35892975]
 [    2016 35892975 41100298]
 [    2017 41100298 47204811]
 [    2018 47204811 53485496]
 [    2019 53485496 60626235]
 [    2020 60626235 66116372]
 [    2021 66116372 66118536]]


In [12]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2020: 100%|██████████| 1/1 [00:00<00:00, 36.09it/s, pdf=142659, year=2020]
2021: 0it [00:00, ?it/s]


In [13]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2001151400,30,3.0,4.0,1.0,6.0,2.0,3.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0
1,2129196167,6219,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2120177826,7033,2.0,4.0,3.0,0.0,1.0,3.0,2.0,3.0,1.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0
3,2015533689,9615,2.0,3.0,0.0,4.0,0.0,1.0,4.0,5.0,3.0,3.0,2.0,3.0,4.0,0.0,0.0,0.0
4,1982860720,11585,9.0,5.0,1.0,6.0,4.0,3.0,4.0,7.0,7.0,2.0,8.0,2.0,2.0,3.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76824,1970437212,2253376,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76825,2252965853,2253384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76826,45876518,2253400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76827,2239425550,2253402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [15]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2001151400,30,1.386294,2.079442,2.197225,2.708050,2.833213,2.995732,3.044523,3.135494,3.258096,3.332205,3.367296,3.401197,3.465736,3.496508,3.526361,3.526361
1,2129196167,6219,1.098612,1.386294,1.609438,1.609438,1.791759,1.791759,1.791759,1.945910,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.197225,2.197225
2,2120177826,7033,1.098612,1.945910,2.302585,2.302585,2.397895,2.639057,2.772589,2.944439,2.995732,3.091043,3.091043,3.178054,3.178054,3.258096,3.258096,3.258096
3,2015533689,9615,1.098612,1.791759,1.791759,2.302585,2.302585,2.397895,2.708050,2.995732,3.135494,3.258096,3.332205,3.433987,3.555348,3.555348,3.555348,3.555348
4,1982860720,11585,2.302585,2.708050,2.772589,3.091043,3.258096,3.367296,3.496508,3.688879,3.850148,3.891820,4.043051,4.077538,4.110874,4.158883,4.219508,4.219508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76824,1970437212,2253376,0.000000,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294
76825,2252965853,2253384,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
76826,45876518,2253400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
76827,2239425550,2253402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))